In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/DomoArigatoData.csv')

In [3]:
def splittodict(line):
    split = line.split(' ; ')
    for i in range(len(split)):
        split[i] = split[i].split(':')
    return dict(split)

In [4]:
df['full_history'][163] = 'consent_approval:yes ; ethnicity:white ; cancer_dx:no ; cancer_dx_type: ; cancer_dx_age: ; known_brca:unknown ; known_cancer:no ; rel_relation:["father", "paternal grandmother", "paternal aunt", "mother"] ; rel_cancer:["pancreas", "pancreas", "pancreas", "squamous cell carcinoma"] ; rel_age:["60", "81", "76", "83"]'

<ipython-input-4-3ab8733b54e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_history'][163] = 'consent_approval:yes ; ethnicity:white ; cancer_dx:no ; cancer_dx_type: ; cancer_dx_age: ; known_brca:unknown ; known_cancer:no ; rel_relation:["father", "paternal grandmother", "paternal aunt", "mother"] ; rel_cancer:["pancreas", "pancreas", "pancreas", "squamous cell carcinoma"] ; rel_age:["60", "81", "76", "83"]'


In [5]:
df['full_history'] = df['full_history'].apply(splittodict)

In [6]:
df = pd.concat([df.drop(['full_history'], axis=1), df['full_history'].apply(pd.Series)], axis=1)

In [7]:
df

,Pathogenic?,Gene,history_class,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,known_cancer,rel_relation,rel_cancer,rel_age,_method,other_cancer,relationships
0,False,NaN,strong_personal,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,yes,"[""Mother"", ""Grandmother Mothers Side"", ""Grandf...","[""Ovarian"", ""Leukemia CML"", ""Bladder "", ""Squam...","[""74"", ""82"", ""35"", ""67"", ""55"", ""64"", ""55""]",NaN,NaN,NaN
1,False,NaN,strong_personal,NaN,White,no,Melanoma,32,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",NaN,NaN,NaN
2,False,NaN,strong_personal,NaN,Hispanic,yes,Breast,37,unknown,yes,"[""Father"", ""Sister"", ""Aunt""]","[""Colon"", ""Uterine "", ""Colon""]","[""74"", ""60"", ""70""]",NaN,NaN,NaN
3,False,NaN,strong_personal,NaN,english extraction caucasion,yes,Prostate,55,unknown,yes,"[""father"", ""uncle/ father's brother"", """"]","[""he died of lymphoma at 86"", ""prostae"", """"]","[""72"", ""not sure"", """"]",NaN,NaN,NaN
4,False,NaN,strong_personal,yes,"white, english, irish",yes,uterine,58,unknown,yes,"[""brother"", ""brother"", ""female cousin"", ""mater...","[""colon"", ""colon"", ""colon"", ""breast"", ""brain"",...","[""60"", ""65"", ""72"", ""80"", ""70"", ""74""]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,yes,White,no,,,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN
2209,False,NaN,none,yes,White,no,,,unknown,no,"[""Grandfather"", """", """"]","[""Lymph node"", """", """"]","[""81"", """", """"]",patch,NaN,NaN
2210,False,NaN,none,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN
2211,False,NaN,none,yes,Caucasian,no,,,no,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN


Cleaning up relationships Column 

In [32]:
relationships = []
for row in pd.DataFrame(df[df['relationships'].notna()]).iterrows():
    relationships.append(row[1]['relationships'])

In [36]:
relationships = [x.replace('=>', ':') for x in relationships]

In [48]:
import json

In [62]:
thefinaldict= {'relationship_type':[], 'cancer_dx_type':[], 'cancer_dx_age' : []}
for i in range(len(relationships)):
    box=json.loads(relationships[i])
    relationshiptype = []
    cancertype = []
    cancerage = []
    for i in range(len(box)):
        for item in box[i].items():
            if item[0]=='relationship_type':
                relationshiptype.append(item[1])
            if item[0]=='cancer_dx_type':
                cancertype.append(item[1])
            if item[0]=='cancer_dx_age':
                cancerage.append(item[1])
    thefinaldict['relationship_type'].append(relationshiptype) 
    thefinaldict['cancer_dx_type'].append(cancertype)
    thefinaldict['cancer_dx_age'].append(cancerage)

In [64]:
pd.DataFrame(thefinaldict)

,relationship_type,cancer_dx_type,cancer_dx_age
0,[Grandmother],[Ovarian ],[55+]
1,"[Sister, Father, Mother, Aunt, paternal (my fa...","[Ovarian Cancer, Lung/shoulders/brain (died ag...","[62, 49, 73, 64, ?, ?]"
2,[mother],[ovarian/colon they were uncertain],[70]
3,[Sister ],[Breast],[33]
4,"[Grandfather Paternal , Grandfather Maternal ]","[Prostate , Bone ]","[, ]"
5,"[aunt, grandfather, uncle, uncle]","[breast, stomach, stomach, stomach]","[60, 60, 50, 50]"
6,"[Brother, Grandmother, Grandmother]","[Luekemia, Breast Cancer, Thyroid cancer]","[5, 50ish, 50ish]"
7,"[Mother, Father, Grandmother]","[Kidney, Bladder, Melanoma]","[52, 45, Unknown]"
8,"[Father, Paternal Great-Grandmother, Paternal ...","[Prostate, Breast, Breast, Chronic Myelogenous...","[60, , , 63]"
9,"[Aunt, Aunt, Grandmother]","[Breast, Breast, Breast]","[50s, Unknown, Unknown]"


Cleaning Ez Columns

In [65]:
df

,Pathogenic?,Gene,history_class,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,known_cancer,rel_relation,rel_cancer,rel_age,_method,other_cancer,relationships
0,False,NaN,strong_personal,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,yes,"[""Mother"", ""Grandmother Mothers Side"", ""Grandf...","[""Ovarian"", ""Leukemia CML"", ""Bladder "", ""Squam...","[""74"", ""82"", ""35"", ""67"", ""55"", ""64"", ""55""]",NaN,NaN,NaN
1,False,NaN,strong_personal,NaN,White,no,Melanoma,32,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",NaN,NaN,NaN
2,False,NaN,strong_personal,NaN,Hispanic,yes,Breast,37,unknown,yes,"[""Father"", ""Sister"", ""Aunt""]","[""Colon"", ""Uterine "", ""Colon""]","[""74"", ""60"", ""70""]",NaN,NaN,NaN
3,False,NaN,strong_personal,NaN,english extraction caucasion,yes,Prostate,55,unknown,yes,"[""father"", ""uncle/ father's brother"", """"]","[""he died of lymphoma at 86"", ""prostae"", """"]","[""72"", ""not sure"", """"]",NaN,NaN,NaN
4,False,NaN,strong_personal,yes,"white, english, irish",yes,uterine,58,unknown,yes,"[""brother"", ""brother"", ""female cousin"", ""mater...","[""colon"", ""colon"", ""colon"", ""breast"", ""brain"",...","[""60"", ""65"", ""72"", ""80"", ""70"", ""74""]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,yes,White,no,,,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN
2209,False,NaN,none,yes,White,no,,,unknown,no,"[""Grandfather"", """", """"]","[""Lymph node"", """", """"]","[""81"", """", """"]",patch,NaN,NaN
2210,False,NaN,none,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN
2211,False,NaN,none,yes,Caucasian,no,,,no,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,NaN,NaN


In [13]:
#cleaning pathogenic
df["Pathogenic?"] = df["Pathogenic?"].astype(int)
df["Pathogenic?"].value_counts(dropna=False)

False    2087
True      126
Name: Pathogenic?, dtype: int64

In [14]:
#cleaning gene
df['Gene'] = df['Gene'].fillna('')
df["Gene"].value_counts(dropna=False)

NaN            2087
MUTYH            27
CHEK2            24
ATM              11
BRCA1             9
BRCA2             9
RAD50             6
PALB2             6
FH                5
NBN               4
BRIP1             3
PMS2              3
MSH6              2
APC               2
CDKN2A            2
VHL               2
RAD51D            2
CHEK2,MUTYH       1
MRE11A            1
MLH1              1
MUTYH,PMS2        1
MSH6,CHEK2        1
NF1               1
BARD1             1
CDH1              1
RAD51C            1
Name: Gene, dtype: int64

In [27]:
#cleaning history_class
df['history_class'].replace('not_strong ', 'not_strong').value_counts(dropna=False)

not_strong         853
strong_family      710
none               631
strong_personal     19
Name: history_class, dtype: int64

In [ ]:
#cleaning consent approval
df['consent_approval'].drop()

In [28]:
#cleaning cancer dx
df['cancer_dx'].replace('Yes','yes').replace('No', 'no').map(dict(yes=1, no=0))
df[df['cancer_dx'].isnull()]['cancer_dx'][399]=1
df['cancer_dx'].fillna(0)

0.0    2115
1.0      94
NaN       4
Name: cancer_dx, dtype: int64

In [84]:
#cleaning _method
df['_method'].drop()

              2071
50               8
51               5
52               5
46               4
              ... 
55's             1
Basal Cell       1
29,31            1
60, 65, 80       1
16               1
Name: cancer_dx_age, Length: 68, dtype: int64

In [86]:
#cleaning known_cancer
df['known_cancer'].drop()


no     1733
yes     312
Name: known_cancer, dtype: int64

In [26]:
#cleaning known_brca
df['known_brca'].fillna('unknown')
df['known_brca'].replace('Yes','yes').replace('No', 'no').replace('U', 'unknown').map(dict(yes=1, no=0, unknown=-1)).value_counts(dropna=False)


-1.0    2032
 0.0      95
 1.0      73
 NaN      13
Name: known_brca, dtype: int64

In [18]:
df['known_brca'].value_counts(dropna=False)

unknown    2020
no           93
yes          72
NaN          13
U            12
No            2
Yes           1
Name: known_brca, dtype: int64